# Train the model

A preprocessed Nettiauto dataset is used to train the XGBoost model. The model and accompanying data is exported to separate files. The API server uses these files.

This notebook needs `nettiauto_dataset.csv` file created by `nettiauto_model.ipynb` notebook.

Running this notebook creates these files:

- `data_columns.txt`
- `model.json`
- `example_X.json`
- `example_y.json`

In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option('display.max_columns', None)


### Hide warnings

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Load preprocessed dataset

The dataset file has been created in the: `nettiauto_dataset_parsing.ipynb`

In [3]:
df = pd.read_csv('./nettiauto_dataset.csv')
# df = df.drop('color', axis=1)

df.head(25)

,make,model,color,driveType,price,totalOwners,kilometers,seats,power,batteryCapacity,age,isSuv,metallicColor,airconditioning,parking_sensors,satellite_navigator,cruise_control_adaptive,parking_camera_simple_camera,seat_heaters,driving_assistant,alloy_wheels,electric_mirrors,lane_departure_warning_system,electrically_operated_tailgate,leather_upholstery,emergency_brake_assist,collision_avoidance_system,cruise_control_traditional,parking_assistant,tow_bar,electric_seats_with_memory,heated_steering_wheel,sunroof,sun_hatch_with_panorama,adaptive_headlights,sport_seats,fuel_battery_powered_heater,parking_camera_360-degree_camera,battery_preheating,electric_seats_without_memory,air_suspension,curve_lights,head_up_display,sport_base
0,porsche,taycan,black,rwd,101880.0,1,2100.0,5,300.0,79.00,2.416667,0,0,1,1,1,0,0,1,0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
1,mercedes-benz,eqs,gray,4wd,125900.0,1,30000.0,5,385.0,108.40,2.000000,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,1,1,1,1,1,0,1,0,0,0
2,nissan,leaf,black,fwd,35800.0,1,10000.0,5,160.0,62.00,2.000000,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,citroen,e-c4,blue,fwd,33800.0,1,17000.0,5,100.0,50.00,2.000000,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,hyundai,kona,white,fwd,27590.0,1,56000.0,5,150.0,64.00,2.750000,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,1,0
5,citroen,e-c4,beige,fwd,29700.0,1,34000.0,5,100.0,50.00,2.000000,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,polestar,2,black,4wd,60780.0,1,7000.0,5,300.0,75.00,2.000000,0,0,1,1,1,1,0,1,1,1,1,0,1,0,1,1,0,1,0,1,1,1,0,1,1,1,1,0,0,0,1,0,0
7,peugeot,e-2008,orange,fwd,25790.0,1,13000.0,5,100.0,50.00,2.500000,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,bmw,ix3,white,rwd,51800.0,1,17000.0,5,210.0,80.00,2.166667,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
9,bmw,ix3,white,rwd,49900.0,1,25500.0,5,210.0,80.00,2.000000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0


## Boolean conversion

- Accessories 0/1 values (present/not-present), such as metallicColor = 1 or 0
- New columns are created for different *Make, model, color* and *driveType* strings and exclusive boolean values given for each

In [4]:
# Change Accessories 0/1 integer values to booleans
df.iloc[:, 11:] = df.iloc[:, 11:].astype(bool)

# Convert to make, model, color and dataType strings to distinct boolean values
df_encoded = pd.get_dummies(df, columns=[
    "make",
    "model",
    "color",
    "driveType"
])

df_encoded.head(25)


,price,totalOwners,kilometers,seats,power,batteryCapacity,age,isSuv,metallicColor,airconditioning,parking_sensors,satellite_navigator,cruise_control_adaptive,parking_camera_simple_camera,seat_heaters,driving_assistant,alloy_wheels,electric_mirrors,lane_departure_warning_system,electrically_operated_tailgate,leather_upholstery,emergency_brake_assist,collision_avoidance_system,cruise_control_traditional,parking_assistant,tow_bar,electric_seats_with_memory,heated_steering_wheel,sunroof,sun_hatch_with_panorama,adaptive_headlights,sport_seats,fuel_battery_powered_heater,parking_camera_360-degree_camera,battery_preheating,electric_seats_without_memory,air_suspension,curve_lights,head_up_display,sport_base,make_audi,make_bmw,make_byd,make_citroen,make_cupra,make_dacia,make_ds,make_fiat,make_ford,make_hyundai,make_jaguar,make_kia,make_lexus,make_maxus,make_mazda,make_mercedes-benz,make_mg,make_mini,make_muu merkki,make_nissan,make_opel,make_peugeot,make_polestar,make_porsche,make_renault,make_seat,make_skoda,make_subaru,make_tesla,make_toyota,make_volkswagen,make_volvo,model_2,model_208,model_3,model_500e,model_ariya,model_atto3,model_born,model_bz4x,model_c4,model_c40,model_citigo,model_combo-e,model_cooper,model_corsa-e,model_e-2008,model_e-208,model_e-308,model_e-berlingo,model_e-c4,model_e-c4 x,model_e-niro,model_e-soul,model_e-spacetourer,model_e-traveller,model_e-tron,model_e-tron gt,model_e-tron gt rs,model_e-up!,model_enyaq,model_eqa,model_eqb,model_eqc,model_eqe,model_eqs,model_eqs suv,model_eqv,model_euniq,model_ev6,model_ev9,model_han,model_i-pace,model_i3,model_i3s,model_i4,model_i4 m50,model_i7,model_id. buzz,model_id3,model_id4,model_id5,model_ioniq 5,model_ioniq 6,model_ioniq electric,model_ix,model_ix m60,model_ix1,model_ix3,model_kona,model_kona electric,model_leaf,model_marvel r,model_megane,model_mg4,model_mg5,model_mifa 9,model_mii electric,model_model 3,model_model s,model_model x,model_model y,model_mokka,model_mokka-e,model_mustang,model_mustang mach-e,model_muu malli,model_mx-30,model_niro,model_proace city verso ev,model_proace verso ev,model_q4 e-tron,model_q8 e-tron,model_solterra,model_spring,model_t90,model_tang,model_taycan,model_twingo,model_up!,model_ux,model_xc40,model_zafira-e,model_zoe,model_zs,color_beige,color_black,color_blue,color_brown,color_copper,color_gray,color_green,color_orange,color_other,color_red,color_silver,color_turquoise,color_violet,color_white,color_yellow,driveType_4wd,driveType_fwd,driveType_rwd
0,101880.0,1,2100.0,5,300.0,79.00,2.416667,False,False,True,True,True,False,False,True,False,True,True,False,True,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,125900.0,1,30000.0,5,385.0,108.40,2.000000,False,False,True,True,True,True,False,True,True,True,True,True,True,True,False,True,False,True,False,True,False,False,True,True,True,True,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

## Model training

### Test split

In [5]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop('price', axis=1)
y = df_encoded['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape

((3184, 182), (796, 182))

### XGBoost fit

In [6]:
import xgboost as xgb

xgb_regressor = xgb.XGBRegressor(objective ='reg:squarederror',
                                 n_estimators=100,
                                 max_depth=8,
                                 reg_lambda=1, # näitä voi säädellä muttei kauheesti oo vaikuttanut malliin
                                 reg_alpha=0,# näitä voi säädellä muttei kauheesti oo vaikuttanut malliin
                                 num_parallel_tree=5, # random forest 🤷‍♂️
)

xgb_regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=5, random_state=None, ...)

### Export the model and feature list

In [7]:
import json


### Export the model
xgb_regressor.save_model("model.json")


### Export feature names
# Feature names from the model
# column_names = list(xgb_regressor.feature_names_in_)

# Feature names from the test data frame

In [8]:
column_names = list(df.drop("price", axis=1).columns)

with open("data_columns.txt", "w") as f:
    f.write("\n".join(name for name in column_names))

### Export an example vector and a prediction

These can be used for testing or in a mock API

In [9]:
# Take the nth single row from X and y dataframes
n = 10

# Save a single row dataframe as json
example_X = X.iloc[[n]]
example_X.to_json("example_X.json", orient="split", index=False)

# Make a prediction and save as json
predicted_y = xgb_regressor.predict(example_X)
example_y = { "price": str(predicted_y[0]) }
with open("example_y.json","w") as f:
    json.dump(example_y, f)



## Test prediction

In [10]:
y_pred = xgb_regressor.predict(X_test)


### The mean absolute error

In [11]:
mean_absolute_error(y_test, y_pred)

3468.453451338725

### The mean squared error

In [12]:
mean_squared_error(y_test, y_pred)

38546047.00740656

### R² score

In [13]:
r2_score(y_test, y_pred)

0.9449857414429049

## Test predictions by make and model

In [14]:
def predict_make_model(make, model):
    m = (X_test[f'make_{make}'] == 1) & (X_test[f'model_{model}'] == 1)
    X_id3 = X_test[m]
    y_id3 = y_test[m]

    y_id3_pred = xgb_regressor.predict(X_id3)

    print('mean_absolute_error: {}'.format(mean_absolute_error(y_id3, y_id3_pred)))
    print('r2_score: {}'.format(r2_score(y_id3, y_id3_pred)))
    print()
    for real, pred in zip(y_id3, y_id3_pred):
        print(f'{real} -> {pred} ~ {real - pred} <> {pred / real}')

### Wolkswagen ID3

In [15]:
predict_make_model('volkswagen', 'id3')

mean_absolute_error: 2165.1561472039475
r2_score: 0.7811816614508794

44540.0 -> 46973.66015625 ~ -2433.66015625 <> 1.0546398777784014
33900.0 -> 27862.904296875 ~ 6037.095703125 <> 0.821914581028761
51500.0 -> 39613.50390625 ~ 11886.49609375 <> 0.7691942506067961
45185.0 -> 46973.66015625 ~ -1788.66015625 <> 1.0395852640533363
44835.0 -> 44812.05859375 ~ 22.94140625 <> 0.9994883147931304
39900.0 -> 43523.2109375 ~ -3623.2109375 <> 1.0908072916666667
37900.0 -> 38719.109375 ~ -819.109375 <> 1.0216123845646439
45900.0 -> 43580.984375 ~ 2319.015625 <> 0.9494767837690632
28390.0 -> 29643.7890625 ~ -1253.7890625 <> 1.044163052571328
46415.0 -> 45566.7265625 ~ 848.2734375 <> 0.9817241530216525
30900.0 -> 31471.890625 ~ -571.890625 <> 1.0185077872168284
39950.0 -> 40589.20703125 ~ -639.20703125 <> 1.0160001760012516
42340.0 -> 43478.328125 ~ -1138.328125 <> 1.0268854068256967
41900.0 -> 39984.6953125 ~ 1915.3046875 <> 0.9542886709427207
45455.0 -> 45680.65234375 ~ -225.65234375 <> 1.00496430

### BMW ix1

In [16]:
predict_make_model('bmw', 'ix1')

mean_absolute_error: 2450.8234375
r2_score: 0.6922714726264216

56900.0 -> 58473.2890625 ~ -1573.2890625 <> 1.0276500713971881
73670.0 -> 78059.8984375 ~ -4389.8984375 <> 1.0595886851839282
65790.0 -> 60827.3125 ~ 4962.6875 <> 0.9245677534579724
69600.0 -> 68659.3828125 ~ 940.6171875 <> 0.986485385237069
65800.0 -> 66187.625 ~ -387.625 <> 1.0058909574468085


### Tesla 3

In [17]:
predict_make_model('tesla', 'model 3')

mean_absolute_error: 2645.8717041015625
r2_score: 0.46236636482477955

39880.0 -> 39756.98046875 ~ 123.01953125 <> 0.9969152574912237
43990.0 -> 42520.24609375 ~ 1469.75390625 <> 0.9665889087008411
39880.0 -> 38764.8125 ~ 1115.1875 <> 0.9720364217652959
42490.0 -> 41564.50390625 ~ 925.49609375 <> 0.9782184962638267
48690.0 -> 52997.0625 ~ -4307.0625 <> 1.0884588724584103
46990.0 -> 47729.640625 ~ -739.640625 <> 1.0157403835922536
42000.0 -> 42266.02734375 ~ -266.02734375 <> 1.006333984375
50790.0 -> 52960.3984375 ~ -2170.3984375 <> 1.0427327906576098
39990.0 -> 37999.28515625 ~ 1990.71484375 <> 0.9502196838272068
52000.0 -> 42902.1015625 ~ 9097.8984375 <> 0.8250404146634616
46800.0 -> 47818.546875 ~ -1018.546875 <> 1.0217638221153846
47690.0 -> 36943.640625 ~ 10746.359375 <> 0.7746622064374082
37990.0 -> 38910.4765625 ~ -920.4765625 <> 1.02422944360358
41000.0 -> 46616.30078125 ~ -5616.30078125 <> 1.1369829458841463
36890.0 -> 34969.71484375 ~ 1920.28515625 <> 0.9479456449918677
54900.

### Opel Mokka-e

In [18]:
predict_make_model('opel', 'mokka-e')

mean_absolute_error: 1997.9212239583333
r2_score: 0.7202658688719341

41990.0 -> 34631.0625 ~ 7358.9375 <> 0.8247454751131221
38960.0 -> 40249.67578125 ~ -1289.67578125 <> 1.03310256112038
44100.0 -> 41131.0234375 ~ 2968.9765625 <> 0.9326762684240363
33490.0 -> 34478.54296875 ~ -988.54296875 <> 1.0295175565467303
31900.0 -> 33851.67578125 ~ -1951.67578125 <> 1.0611810589733541
36990.0 -> 38042.64453125 ~ -1052.64453125 <> 1.0284575434238983
40970.0 -> 40938.46484375 ~ 31.53515625 <> 0.9992302866426653
34900.0 -> 33438.4453125 ~ 1461.5546875 <> 0.958121642191977
40970.0 -> 40938.46484375 ~ 31.53515625 <> 0.9992302866426653
42480.0 -> 39950.80859375 ~ 2529.19140625 <> 0.9404615958980697
27900.0 -> 28606.2109375 ~ -706.2109375 <> 1.0253122199820788
39560.0 -> 43871.23828125 ~ -4311.23828125 <> 1.1089797341064207
41900.0 -> 40938.46484375 ~ 961.53515625 <> 0.9770516669152745
38470.0 -> 39950.80859375 ~ -1480.80859375 <> 1.0384925550753834
27250.0 -> 30094.755859375 ~ -2844.755859375 <> 1.1

### Audi e-tron

In [19]:
predict_make_model('audi', 'e-tron')

mean_absolute_error: 12333.9287109375
r2_score: -2.569044495340448

51800.0 -> 54726.19921875 ~ -2926.19921875 <> 1.0564903324083013
78800.0 -> 53121.58984375 ~ 25678.41015625 <> 0.6741318508090102
75800.0 -> 71131.4765625 ~ 4668.5234375 <> 0.93840998103562
79800.0 -> 71685.6953125 ~ 8114.3046875 <> 0.8983169838659147
49900.0 -> 53141.375 ~ -3241.375 <> 1.0649574148296592
64999.0 -> 61614.8515625 ~ 3384.1484375 <> 0.9479353768904137
63000.0 -> 113146.625 ~ -50146.625 <> 1.7959781746031747
67800.0 -> 68311.84375 ~ -511.84375 <> 1.0075493178466077
